# AutoGrad : 自动求导  
torch.autograd是pytorch自动求导的工具，也是所有神经网络的核心。我们首先先简单了解一下这个包如何训练神经网络。

## 背景介绍  

神经网络(NNs)是作用在输入数据上的一系列嵌套函数的集合，这些函数由权重和误差来定义，被存储在PyTorch中的tensors中。

神经网络训练的两个步骤：

**前向传播：** 在前向传播中，神经网络通过将接收到的数据与每一层对应的权重和误差进行运算来对正确的输出做出最好的预测。

**反向传播：** 在反向传播中，神经网络调整其参数使得其与输出误差成比例。反向传播基于梯度下降策略，是链式求导法则的一个应用，以目标的负梯度方向对参数进行调整。

更加详细的介绍可以参照下述地址：[video from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)



## 在Pytorch 中的应用  

来看一个简单的示例，我们从torchvision加载一个预先训练好的resnet18模型，接着创建一个随机数据tensor来表示一有3个通道、高度和宽度为64的图像，其对应的标签初始化为一些随机值。

In [1]:
%matplotlib inline

In [3]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

接下来，我们将输入数据向输出方向传播到模型的每一层中来预测输出，这就是**前向传播** 。

In [4]:
prediction = model(data) # 前向传播

我们利用模型的预测和对应的标签来计算误差（loss），然后反向传播误差。完成计算后，您可以调用`.backward()`并自动计算所有梯度。此张量的梯度将累积到`.grad`属性中。

In [5]:
loss = (prediction - labels).sum()
loss.backward() # 反向传播

接着，我们加载一个优化器，在本例中，SGD的学习率为0.01，momentum 为0.9。我们在优化器中注册模型的所有参数。

In [6]:
optim = torch.optim.SGD(model.parameters(), lr= 0.001, momentum= 0.9)

最后，我们调用`.step()`来执行梯度下降，优化器通过存储在`.grad`中的梯度来调整每个参数。

In [7]:
optim.step() # 梯度下降

现在，你已经具备了训练神经网络所需所有条件。下面几节详细介绍了Autograd包的工作原理——可以跳过它们。


-----

## Autograd中的求导  

先来看一下autograd是如何收集梯度的。我们创建两个张量a和b并设置requires_grad = True以跟踪它的计算。

In [9]:
import torch

In [15]:
a = torch.tensor([2., 3.], requires_grad= True)
b = torch.tensor([6., 4.], requires_grad= True)

接着在a和b的基础上创建张量Q

\begin{align}Q = 3a^3 - b^2\end{align}

In [17]:
Q = 3*a**3 - b**2

假设a和b是一个神经网络的权重，Q是它的误差，在神经网络训练中，我们需要w.r.t参数的误差梯度，即  

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}

当我们调用Q的.backward()时，autograd计算这些梯度并把它们存储在张量的 .grad属性中。

我们需要在Q.backward()中显式传递gradient，gradient是一个与Q相同形状的张量，它表示Q w.r.t本身的梯度，即  

\begin{align}\frac{dQ}{dQ} = 1\end{align}

同样，我们也可以将Q聚合为一个标量并隐式向后调用，如Q.sum().backward()。  


In [18]:
external_grad = torch.tensor([1., 1,])
Q.backward(gradient=external_grad)

现在梯度都被存放在a.grad和b.grad中

In [19]:
# 检查一下存储的梯度都正确
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


## 选读----用autograd进行向量计算  

在数学上，如果你有一个向量值函数𝑦⃗ =𝑓(𝑥⃗ ) ，则𝑦⃗ 相对于𝑥⃗ 的梯度是雅可比矩阵：  

\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}
      

一般来说，torch.autograd是一个计算雅可比向量积的引擎。 也就是说，给定任何向量𝑣=(𝑣1𝑣2...𝑣𝑚)𝑇，计算乘积$J^{T}\cdot \vec{v}$。  

如果𝑣恰好是标量函数的梯度𝑙=𝑔(𝑦⃗ )，即
 
\begin{align}\vec{v}
   =
   \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}\end{align}
 
 然后根据链式法则，雅可比向量乘积将是𝑙相对于𝑥⃗ 的梯度  
 
\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)\end{align}
      
      
      
      
雅可比向量积的这种特性使得将外部梯度馈送到具有非标量输出的模型中非常方便。external_grad 代表$\vec{v}$
.